In [2]:
import gdal
import cv2
from osgeo import *
from gdal import *

image_data = gdal.Open("/Users/xuyameng/desktop/sc_0726_jf/LC08_L1TP_128038_20140806_20170420_01_T1_merge_align.tif")
final_img = cv2.imread("/Users/xuyameng/Desktop/sc_0722_jf/he.png")
print(final_img.shape)
# final_img = image_data.ReadAsArray()
driver = image_data.GetDriver()
out_dataset=driver.Create("/Users/xuyameng/Desktop/sc_0726_jf/cpc.tif",final_img.shape[1], final_img.shape[0],1, GDT_Int32)
for i in range(1):
    outBand = out_dataset.GetRasterBand(i + 1)
    outBand.WriteArray(final_img[:,:,i])
out_dataset.SetGeoTransform(image_data.GetGeoTransform())
out_dataset.SetProjection(image_data.GetProjection())
del out_dataset

(7761, 7601, 3)


In [3]:
import geopandas as gpd
import numpy as np
import gdal
import cv2
from osgeo.gdalconst import *
import osgeo
from pathlib import Path
def geo2imagexy(dataset: osgeo.gdal.Dataset, x: float, y: float) -> str:
    """
    parameters:
        dataset: osgeo.gdal.Dataset
            GDAL geographic data
        x: float
            Projection or geographic coordinates x
        y: float
            Projection or geographic coordinates y
            
    return :float
        A line or column number on an image corresponding(x,y) to a shadow or geographic coordinate(row, col) 
    """
    trans = dataset.GetGeoTransform()
    a = np.array([[trans[1], trans[2]], [trans[4], trans[5]]])
    b = np.array([x - trans[0], y - trans[3]])
    return np.linalg.solve(a, b)  # 使用numpy的linalg.solve进行二元一次方程的求解
    
def shp2tif(shape_path: Path, tif_path: Path,  pixel: str) -> np.ndarray:
    """
    parameters:
        shape_path: Path
            input path of shape file
        tif_path: Path
            input path of tif file
        pixel:
            the pixel value after shape2tif
    return:
        all_array: np.ndarray
            array after shape2tif
        dataset: osgeo.gdal.Dataset
            GDAL geographic data
    """
    global dataset
    dataset = gdal.Open(path_tif)
    global rows,cols
    rows = dataset.RasterYSize
    cols = dataset.RasterXSize
    geo_data=gpd.read_file(path_shp)
    all_array=np.zeros((rows,cols),dtype=np.uint8)
    shp_lists = []
    shape_list = []
    for i in list(geo_data.geometry):
        if i.type == 'Polygon':
            temp_list=list(i.exterior.coords)  #获取polygon外环点坐标
            temp_point_list = []
            for point in temp_list:
                [point_x,point_y] = geo2imagexy(dataset,point[0],point[1])  # 求解shape点坐标对应的tif坐标
                temp_point_list.append([point_x,point_y])
            shp_lists.append(temp_point_list)
            
            interiors_list = list(i.interiors)  # 找到polygon内环
            for interiors in interiors_list:
                interiors_temp_list = list(interiors.coords)  # 获取内环的点坐标 
                interiors_point_list = []
                for interiors_point in interiors_temp_list:
                    [interiors_point_x,interiors_point_y] = geo2imagexy(dataset,interiors_point[0],interiors_point[1])
                    interiors_point_list.append([interiors_point_x,interiors_point_y])
                shape_list.append(interiors_point_list)
        else:
            print(i.type)
    for i in shp_lists:
        poly = np.array(i, np.int32)
        cv2.fillPoly(all_array,[poly], pixel)  # 外环点用指定值填充
    for k in shape_list:
        shape_poly = np.array(k,np.int32) # 内环用0填充
        cv2.fillPoly(all_array,[shape_poly],0) 
    print("shape2tif successful:")
    return all_array,dataset
def save_tif(out_path: Path , all_array: np.ndarray, dataset: osgeo.gdal.Dataset):
    """
    parameters:
        out_path: Path 
            output path of tif file
        all_array: np.ndarray
            Prepare the array to store
        dataset: osgeo.gdal.Dataset
            GDAL geographic data
    """
    driver = dataset.GetDriver()
    out_dataset = driver.Create(out_path,all_array.shape[1], all_array.shape[0],1, GDT_UInt16)
    outBand = out_dataset.GetRasterBand(1)
    outBand.WriteArray(all_array[:,:])
    out_dataset.SetGeoTransform(dataset.GetGeoTransform())
    out_dataset.SetProjection(dataset.GetProjection())
    print("save tif file successful")
    del out_dataset   
if __name__ == '__main__':
    path_tif = "/Users/wanggang/Desktop/jianfengbiaozhu/LC81210402015156LGN00_merge.tif"
    path_shp = "/Users/wanggang/Downloads/cloud_cheng/cloud_cheng.shp"
    out_path = "/Users/wanggang/Desktop/2015/get.tif"
    rgb_values = 2
    all_array,dataset = shp2tif(path_shp,path_tif,rgb_values)
    save_tif(out_path,all_array,dataset)

AttributeError: 'NoneType' object has no attribute 'RasterYSize'

In [8]:
path_tif = "/Users/wanggang/Desktop/jianfengbiaozhu/LC81210402015156LGN00_merge.tif"
path_shp = "/Users/wanggang/Downloads/cloud_cheng/cloud_cheng.shp"

In [9]:
shape_data = gpd.read_file(path_shp)